In [1]:
## Import All Needed Libraries

import os
import gym
import ray
from animalai.envs.arena_config import ArenaConfig
from animalai.envs.gym.environment import AnimalAIGym
from ray.rllib.agents import ppo

from ray.tune import register_env, tune

from ray.rllib.agents.ppo import PPOTrainer
from ray.rllib.models import ModelCatalog
from ray.tune.logger import pretty_print

from cache_model import *
from config import get_cfg
from custom_model import *

Instructions for updating:
non-resource variables are not supported in the long term
2021-04-05 15:23:44,888	WARNING deprecation.py:34 -- DeprecationWarning: `ray.rllib.env.atari_wrappers....` has been deprecated. Use `ray.rllib.env.wrappers.atari_wrappers....` instead. This will raise an error in the future!


In [2]:
## Reuse Wrapper for AnimalAI Environment

class UnityEnvWrapper(gym.Env):
    def __init__(self, env_config):
        self.vector_index = env_config.vector_index
        self.worker_index = env_config.worker_index
        self.worker_id = env_config["unity_worker_id"] + env_config.worker_index
        self.env = AnimalAIGym(
            environment_filename = "../examples/env/AnimalAI",
            worker_id = self.worker_id,
            flatten_branched = True,
            uint8_visual = True,
            arenas_configurations = ArenaConfig(env_config['arena_to_train'])
        )
        self.action_space = self.env.action_space
        self.observation_space = self.env.observation_space
        
    def reset(self):
        return self.env.reset()
    
    def step(self, action):
        return self.env.step(action)

In [3]:
arena_configurations = ['0.yml', '1.yml', '2.yml', '3.yml', '4.yml']

result_per_arena = [0.8, 0.8, 0.8, 0.6, 0.2]
MIN_NUMBER_OF_EPISODES = 100
checkpoint = ""

for item in range(len(arena_configurations)):
    
    ## Setup configuration to use
    conf = {
        "num_workers": 0,
        "env_config": {
            "unity_worker_id": 60,
            "arena_to_train": '../examples/configurations/curriculum/' + arena_configurations[item],
            },
            "model": {
                "custom_model": 'my_cnn_rnn_model',
                "custom_model_config": {},
            },
            "num_gpus": int(os.environ.get("RLLIB_NUM_GPUS", "2")),
            "num_workers": 1,  # parallelism
            "framework": "torch",
            "train_batch_size": 500
           }
    
    
    ## Setup and register environment
    ray.shutdown()
    ray.init()

    # Register custom models so that we can give the ID to the policy trainer
    # ModelCatalog.register_custom_model("my_fc_model", MyFCForwardModel)
    ModelCatalog.register_custom_model("my_rnn_model", MyRNNModel)
    ModelCatalog.register_custom_model("my_convgru_model", MyConvGRUModel)  # NOTE: Only works with image observations.
    ModelCatalog.register_custom_model("my_cnn_rnn_model", MyCNNRNNModel)

    register_env("unity_env", lambda config: UnityEnvWrapper(config))
    
    ## Setup trainer
    trainer = PPOTrainer(config=conf, env= "unity_env")
    
    ## Loading and restoring a trained agent from a checkpoint 
    if checkpoint:    
        trainer.restore(checkpoint)
    
    result = trainer.train()
    min_episodes = MIN_NUMBER_OF_EPISODES
    
    ## Train the model until performance improves to a certain threshold
    while result['episode_reward_mean'] < result_per_arena[item] and min_episodes > 0 and not result['done']:
        result = trainer.train()
        min_episodes -= 1

    checkpoint = trainer.save()
    print("checkpoint saved at", checkpoint)
    print(pretty_print(result))

2021-04-05 15:24:51,950	INFO services.py:1174 -- View the Ray dashboard at http://127.0.0.1:8266
2021-04-05 15:24:53,610	INFO trainer.py:643 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=54642) WARNING:tensorflow:From /home/azibit/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/compat/v2_compat.py:68: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=54642) Instructions for updating:
(pid=54642) non-resource variables are not supported in the long term


(pid=54642) Found path: /home/azibit/COMS_673/AnimalAI-Olympics/cachey/../examples/env/AnimalAI.x86_64
(pid=54642) Mono path[0] = '/home/azibit/COMS_673/AnimalAI-Olympics/cachey/../examples/env/AnimalAI_Data/Managed'
(pid=54642) Mono config path = '/home/azibit/COMS_673/AnimalAI-Olympics/cachey/../examples/env/AnimalAI_Data/MonoBleedingEdge/etc'
(pid=54642) Preloaded 'lib_burst_generated.so'
(pid=54642) Preloaded 'libgrpc_csharp_ext.x64.so'
(pid=54642) Preloaded 'ScreenSelector.so'
(pid=54642) Display 0 '0': 1920x1080 (primary device).
(pid=54642) Display 1 'C32F391 32"': 1920x1080 (secondary device).
(pid=54642) Logging to /home/azibit/.config/unity3d/Unity Technologies/UnityEnvironment/Player.log


(pid=54642) INFO:mlagents_envs:Connected to Unity environment with package version 0.15.0-preview and communication version 0.15.0
(pid=54642) INFO:mlagents_envs:Connected new brain:
(pid=54642) AnimalAI?team=0
(pid=54642) INFO:gym_unity:1 agents within environment.
(pid=54642) 2021-04-05 15:24:59,241	WARNING deprecation.py:34 -- DeprecationWarning: `framestack` has been deprecated. Use `num_framestacks (int)` instead. This will raise an error in the future!
(pid=54642) 2021-04-05 15:24:59,282	WARNING deprecation.py:34 -- DeprecationWarning: `ray.rllib.env.atari_wrappers....` has been deprecated. Use `ray.rllib.env.wrappers.atari_wrappers....` instead. This will raise an error in the future!
2021-04-05 15:24:59,375	WARNING deprecation.py:34 -- DeprecationWarning: `framestack` has been deprecated. Use `num_framestacks (int)` instead. This will raise an error in the future!
2021-04-05 15:24:59,475	WARNING util.py:47 -- Install gputil for GPU system monitoring.
(pid=54642) 2021-04-05 15:2

checkpoint saved at /home/azibit/ray_results/PPO_unity_env_2021-04-05_15-24-53jft6kf3m/checkpoint_3/checkpoint-3
custom_metrics: {}
date: 2021-04-05_15-26-11
done: false
episode_len_mean: 35.46
episode_reward_max: 2.1800550371408463
episode_reward_mean: 0.823587782047689
episode_reward_min: -0.9999999310821295
episodes_this_iter: 31
episodes_total: 50
experiment_id: 8282e29d6bd049c5b9713d59cbf0a763
hostname: azibit-Lenovo-Y520-15IKBM
info:
  learner:
    default_policy:
      allreduce_latency: 0.0
      cur_kl_coeff: 0.45000000000000007
      cur_lr: 5.0e-05
      entropy: 2.06179416179657
      entropy_coeff: 0.0
      kl: 0.023325731977820396
      policy_loss: -0.18025227123871446
      total_loss: -0.12271115649491549
      vf_explained_var: 0.7569695115089417
      vf_loss: 0.04704452259466052
  num_steps_sampled: 1800
  num_steps_trained: 1800
iterations_since_restore: 3
node_ip: 192.168.1.4
num_healthy_workers: 1
off_policy_estimator: {}
perf:
  cpu_util_percent: 38.52
  ram_ut

2021-04-05 15:26:13,985	INFO services.py:1174 -- View the Ray dashboard at http://127.0.0.1:8266
(pid=55272) WARNING:tensorflow:From /home/azibit/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/compat/v2_compat.py:68: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=55272) Instructions for updating:
(pid=55272) non-resource variables are not supported in the long term
(pid=55272) WARNING:tensorflow:From /home/azibit/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/compat/v2_compat.py:68: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=55272) Instructions for updating:
(pid=55272) non-resource variables are not supported in the long term


...

checkpoint saved at /home/azibit/ray_results/PPO_unity_env_2021-04-05_17-03-40pucr37ea/checkpoint_306/checkpoint-306
custom_metrics: {}
date: 2021-04-05_17-38-28
done: false
episode_len_mean: 245.87
episode_reward_max: 0.489666691981256
episode_reward_mean: -0.9034265980590135
episode_reward_min: -0.9999999310821295
episodes_this_iter: 2
episodes_total: 1047
experiment_id: 8282e29d6bd049c5b9713d59cbf0a763
hostname: azibit-Lenovo-Y520-15IKBM
info:
  learner:
    default_policy:
      allreduce_latency: 0.0
      cur_kl_coeff: 1.5187500000000003
      cur_lr: 5.0e-05
      entropy: 1.667035174369812
      entropy_coeff: 0.0
      kl: 0.013021756522357463
      policy_loss: -0.09588294923305511
      total_loss: -0.07088536769151688
      vf_explained_var: 0.4998610019683838
      vf_loss: 0.0052207885310053825
  num_steps_sampled: 183600
  num_steps_trained: 183600
iterations_since_restore: 101
node_ip: 192.168.1.4
num_healthy_workers: 1
off_policy_estimator: {}
perf:
  cpu_util_percent:

2021-04-05 17:38:31,053	INFO services.py:1174 -- View the Ray dashboard at http://127.0.0.1:8266
(pid=68242) WARNING:tensorflow:From /home/azibit/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/compat/v2_compat.py:68: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=68242) Instructions for updating:
(pid=68242) non-resource variables are not supported in the long term
(pid=68242) WARNING:tensorflow:From /home/azibit/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/compat/v2_compat.py:68: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=68242) Instructions for updating:
(pid=68242) non-resource variables are not supported in the long term
(pid=68242) WARNING:tensorflow:From /home/azibit/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/compat/v2_compat.py:68: disable_resource_variables (fro

(pid=68242) Found path: /home/azibit/COMS_673/AnimalAI-Olympics/cachey/../examples/env/AnimalAI.x86_64
(pid=68242) Mono path[0] = '/home/azibit/COMS_673/AnimalAI-Olympics/cachey/../examples/env/AnimalAI_Data/Managed'
(pid=68242) Mono config path = '/home/azibit/COMS_673/AnimalAI-Olympics/cachey/../examples/env/AnimalAI_Data/MonoBleedingEdge/etc'
(pid=68242) Found path: /home/azibit/COMS_673/AnimalAI-Olympics/cachey/../examples/env/AnimalAI.x86_64
(pid=68242) Mono path[0] = '/home/azibit/COMS_673/AnimalAI-Olympics/cachey/../examples/env/AnimalAI_Data/Managed'
(pid=68242) Mono config path = '/home/azibit/COMS_673/AnimalAI-Olympics/cachey/../examples/env/AnimalAI_Data/MonoBleedingEdge/etc'
(pid=68242) Found path: /home/azibit/COMS_673/AnimalAI-Olympics/cachey/../examples/env/AnimalAI.x86_64
(pid=68242) Mono path[0] = '/home/azibit/COMS_673/AnimalAI-Olympics/cachey/../examples/env/AnimalAI_Data/Managed'
(pid=68242) Mono config path = '/home/azibit/COMS_673/AnimalAI-Olympics/cachey/../examp

(pid=68242) INFO:mlagents_envs:Connected to Unity environment with package version 0.15.0-preview and communication version 0.15.0
(pid=68242) INFO:mlagents_envs:Connected to Unity environment with package version 0.15.0-preview and communication version 0.15.0
(pid=68242) INFO:mlagents_envs:Connected to Unity environment with package version 0.15.0-preview and communication version 0.15.0
(pid=68242) INFO:mlagents_envs:Connected to Unity environment with package version 0.15.0-preview and communication version 0.15.0
(pid=68242) INFO:mlagents_envs:Connected to Unity environment with package version 0.15.0-preview and communication version 0.15.0
(pid=68242) INFO:mlagents_envs:Connected new brain:
(pid=68242) AnimalAI?team=0
(pid=68242) INFO:gym_unity:1 agents within environment.
(pid=68242) 2021-04-05 17:38:37,086	WARNING deprecation.py:34 -- DeprecationWarning: `framestack` has been deprecated. Use `num_framestacks (int)` instead. This will raise an error in the future!
(pid=68242) I

checkpoint saved at /home/azibit/ray_results/PPO_unity_env_2021-04-05_17-38-31ajn4r3p6/checkpoint_407/checkpoint-407
custom_metrics: {}
date: 2021-04-05_18-15-04
done: false
episode_len_mean: 248.8
episode_reward_max: 0.481666692532599
episode_reward_mean: -0.9851832648459822
episode_reward_min: -0.9999999310821295
episodes_this_iter: 2
episodes_total: 1290
experiment_id: 8282e29d6bd049c5b9713d59cbf0a763
hostname: azibit-Lenovo-Y520-15IKBM
info:
  learner:
    default_policy:
      allreduce_latency: 0.0
      cur_kl_coeff: 2.278125
      cur_lr: 5.0e-05
      entropy: 1.5851064920425415
      entropy_coeff: 0.0
      kl: 0.012575673684477807
      policy_loss: -0.10114725157618523
      total_loss: -0.06987666636705399
      vf_explained_var: 0.6344132423400879
      vf_loss: 0.0026216533384285866
  num_steps_sampled: 244200
  num_steps_trained: 244200
iterations_since_restore: 101
node_ip: 192.168.1.4
num_healthy_workers: 1
off_policy_estimator: {}
perf:
  cpu_util_percent: 26.137837

In [5]:
## Training for Object permanence and spatial reasoning
arena_configurations = ['1_behind.yml', '2_inside.yml', '3_where_is_object.yml', 
                        '4_previously_seen_object.yml', '5_inside_or_behind.yml', '6_spatial_reasoning.yml']

result_per_arena = [0.8, 0.8, 0.8, 0.7, 0.7, 0.6]
MIN_NUMBER_OF_EPISODES = 100
checkpoint = "/home/azibit/ray_results/PPO_unity_env_2021-04-05_17-38-31ajn4r3p6/checkpoint_407/checkpoint-407"

for item in range(len(arena_configurations)):
    
    ## Setup configuration to use
    conf = {
        "num_workers": 0,
        "env_config": {
            "unity_worker_id": 60,
            "arena_to_train": '../examples/configurations/curriculum/' + arena_configurations[item],
            },
            "model": {
                "custom_model": 'my_cnn_rnn_model',
                "custom_model_config": {},
            },
            "num_gpus": int(os.environ.get("RLLIB_NUM_GPUS", "2")),
            "num_workers": 1,  # parallelism
            "framework": "torch",
            "train_batch_size": 500
           }
    
    
    ## Setup and register environment
    ray.shutdown()
    ray.init()

    # Register custom models so that we can give the ID to the policy trainer
    # ModelCatalog.register_custom_model("my_fc_model", MyFCForwardModel)
    ModelCatalog.register_custom_model("my_rnn_model", MyRNNModel)
    ModelCatalog.register_custom_model("my_convgru_model", MyConvGRUModel)  # NOTE: Only works with image observations.
    ModelCatalog.register_custom_model("my_cnn_rnn_model", MyCNNRNNModel)

    register_env("unity_env", lambda config: UnityEnvWrapper(config))
    
    ## Setup trainer
    trainer = PPOTrainer(config=conf, env= "unity_env")
    
    ## Loading and restoring a trained agent from a checkpoint 
    if checkpoint:    
        trainer.restore(checkpoint)
    
    result = trainer.train()
    min_episodes = MIN_NUMBER_OF_EPISODES
    
    ## Train the model until performance improves to a certain threshold
    while result['episode_reward_mean'] < result_per_arena[item] and min_episodes > 0 and not result['done']:
        result = trainer.train()
        min_episodes -= 1

    checkpoint = trainer.save()
    print("checkpoint saved at", checkpoint)
    print(pretty_print(result))

2021-04-05 22:36:21,124	INFO services.py:1174 -- View the Ray dashboard at http://127.0.0.1:8266
(pid=90739) WARNING:tensorflow:From /home/azibit/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/compat/v2_compat.py:68: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=90739) Instructions for updating:
(pid=90739) non-resource variables are not supported in the long term
(pid=90739) WARNING:tensorflow:From /home/azibit/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/compat/v2_compat.py:68: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=90739) Instructions for updating:
(pid=90739) non-resource variables are not supported in the long term
(pid=90739) WARNING:tensorflow:From /home/azibit/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/compat/v2_compat.py:68: disable_resource_variables (fro

(pid=90739) Found path: /home/azibit/COMS_673/AnimalAI-Olympics/cachey/../examples/env/AnimalAI.x86_64
(pid=90739) Mono path[0] = '/home/azibit/COMS_673/AnimalAI-Olympics/cachey/../examples/env/AnimalAI_Data/Managed'
(pid=90739) Mono config path = '/home/azibit/COMS_673/AnimalAI-Olympics/cachey/../examples/env/AnimalAI_Data/MonoBleedingEdge/etc'
(pid=90739) Preloaded 'lib_burst_generated.so'
(pid=90739) Preloaded 'libgrpc_csharp_ext.x64.so'
(pid=90739) Preloaded 'ScreenSelector.so'
(pid=90739) Display 0 '0': 1920x1080 (primary device).
(pid=90739) Display 1 'C32F391 32"': 1920x1080 (secondary device).
(pid=90739) Logging to /home/azibit/.config/unity3d/Unity Technologies/UnityEnvironment/Player.log
(pid=90739) Found path: /home/azibit/COMS_673/AnimalAI-Olympics/cachey/../examples/env/AnimalAI.x86_64
(pid=90739) Mono path[0] = '/home/azibit/COMS_673/AnimalAI-Olympics/cachey/../examples/env/AnimalAI_Data/Managed'
(pid=90739) Mono config path = '/home/azibit/COMS_673/AnimalAI-Olympics/cac

(pid=90739) INFO:mlagents_envs:Connected to Unity environment with package version 0.15.0-preview and communication version 0.15.0
(pid=90739) INFO:mlagents_envs:Connected to Unity environment with package version 0.15.0-preview and communication version 0.15.0
(pid=90739) INFO:mlagents_envs:Connected to Unity environment with package version 0.15.0-preview and communication version 0.15.0
(pid=90739) INFO:mlagents_envs:Connected to Unity environment with package version 0.15.0-preview and communication version 0.15.0
(pid=90739) INFO:mlagents_envs:Connected to Unity environment with package version 0.15.0-preview and communication version 0.15.0
(pid=90739) INFO:mlagents_envs:Connected to Unity environment with package version 0.15.0-preview and communication version 0.15.0
(pid=90739) INFO:mlagents_envs:Connected to Unity environment with package version 0.15.0-preview and communication version 0.15.0
(pid=90739) INFO:mlagents_envs:Connected new brain:
(pid=90739) AnimalAI?team=0
(pi

checkpoint saved at /home/azibit/ray_results/PPO_unity_env_2021-04-05_22-36-21xmetbcrb/checkpoint_508/checkpoint-508
custom_metrics: {}
date: 2021-04-05_23-13-43
done: false
episode_len_mean: 130.03
episode_reward_max: 0.7923333747312427
episode_reward_mean: 0.20009337390773
episode_reward_min: -0.9999999310821295
episodes_this_iter: 5
episodes_total: 1625
experiment_id: 8282e29d6bd049c5b9713d59cbf0a763
hostname: azibit-Lenovo-Y520-15IKBM
info:
  learner:
    default_policy:
      allreduce_latency: 0.0
      cur_kl_coeff: 2.278125
      cur_lr: 5.0e-05
      entropy: 1.5507797300815582
      entropy_coeff: 0.0
      kl: 0.013586960267275572
      policy_loss: -0.19848722219467163
      total_loss: -0.09353464841842651
      vf_explained_var: 0.024836093187332153
      vf_loss: 0.0739997299388051
  num_steps_sampled: 304800
  num_steps_trained: 304800
iterations_since_restore: 101
node_ip: 192.168.1.4
num_healthy_workers: 1
off_policy_estimator: {}
perf:
  cpu_util_percent: 19.29333333

2021-04-05 23:13:45,997	INFO services.py:1174 -- View the Ray dashboard at http://127.0.0.1:8266
(pid=95742) WARNING:tensorflow:From /home/azibit/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/compat/v2_compat.py:68: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=95742) Instructions for updating:
(pid=95742) non-resource variables are not supported in the long term
(pid=95742) WARNING:tensorflow:From /home/azibit/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/compat/v2_compat.py:68: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=95742) Instructions for updating:
(pid=95742) non-resource variables are not supported in the long term
(pid=95742) WARNING:tensorflow:From /home/azibit/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/compat/v2_compat.py:68: disable_resource_variables (fro

...

2021-04-06 00:56:09,257	INFO services.py:1174 -- View the Ray dashboard at http://127.0.0.1:8266
(pid=110249) WARNING:tensorflow:From /home/azibit/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/compat/v2_compat.py:68: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=110249) Instructions for updating:
(pid=110249) non-resource variables are not supported in the long term
(pid=110249) WARNING:tensorflow:From /home/azibit/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/compat/v2_compat.py:68: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=110249) Instructions for updating:
(pid=110249) non-resource variables are not supported in the long term
(pid=110249) WARNING:tensorflow:From /home/azibit/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/compat/v2_compat.py:68: disable_resource_variabl

(pid=110249) Found path: /home/azibit/COMS_673/AnimalAI-Olympics/cachey/../examples/env/AnimalAI.x86_64
(pid=110249) Mono path[0] = '/home/azibit/COMS_673/AnimalAI-Olympics/cachey/../examples/env/AnimalAI_Data/Managed'
(pid=110249) Mono config path = '/home/azibit/COMS_673/AnimalAI-Olympics/cachey/../examples/env/AnimalAI_Data/MonoBleedingEdge/etc'
(pid=110249) Preloaded 'lib_burst_generated.so'
(pid=110249) Preloaded 'libgrpc_csharp_ext.x64.so'
(pid=110249) Preloaded 'ScreenSelector.so'
(pid=110249) Display 0 '0': 1920x1080 (primary device).
(pid=110249) Display 1 'C32F391 32"': 1920x1080 (secondary device).
(pid=110249) Logging to /home/azibit/.config/unity3d/Unity Technologies/UnityEnvironment/Player.log
(pid=110249) Found path: /home/azibit/COMS_673/AnimalAI-Olympics/cachey/../examples/env/AnimalAI.x86_64
(pid=110249) Mono path[0] = '/home/azibit/COMS_673/AnimalAI-Olympics/cachey/../examples/env/AnimalAI_Data/Managed'
(pid=110249) Mono config path = '/home/azibit/COMS_673/AnimalAI-

(pid=110249) INFO:mlagents_envs:Connected to Unity environment with package version 0.15.0-preview and communication version 0.15.0
(pid=110249) INFO:mlagents_envs:Connected to Unity environment with package version 0.15.0-preview and communication version 0.15.0
(pid=110249) INFO:mlagents_envs:Connected to Unity environment with package version 0.15.0-preview and communication version 0.15.0
(pid=110249) INFO:mlagents_envs:Connected to Unity environment with package version 0.15.0-preview and communication version 0.15.0
(pid=110249) INFO:mlagents_envs:Connected to Unity environment with package version 0.15.0-preview and communication version 0.15.0
(pid=110249) INFO:mlagents_envs:Connected to Unity environment with package version 0.15.0-preview and communication version 0.15.0
(pid=110249) INFO:mlagents_envs:Connected to Unity environment with package version 0.15.0-preview and communication version 0.15.0
(pid=110249) INFO:mlagents_envs:Connected to Unity environment with package 

2021-04-06 00:56:15,524	WARNING util.py:47 -- Install gputil for GPU system monitoring.
2021-04-06 00:56:15,541	INFO trainable.py:372 -- Restored on 192.168.1.4 from checkpoint: /home/azibit/ray_results/PPO_unity_env_2021-04-06_00-20-28pxxm9n22/checkpoint_812/checkpoint-812
2021-04-06 00:56:15,541	INFO trainable.py:379 -- Current state after restoring: {'_iteration': 812, '_timesteps_total': None, '_time_total': 18493.03905749321, '_episodes_total': 6456}
(pid=110249) 2021-04-06 00:56:15,567	WARNING deprecation.py:34 -- DeprecationWarning: `env_index` has been deprecated. Use `episode.env_id` instead. This will raise an error in the future!
(pid=110249) 2021-04-06 00:56:15,567	WARNING deprecation.py:34 -- DeprecationWarning: `env_index` has been deprecated. Use `episode.env_id` instead. This will raise an error in the future!
(pid=110249) 2021-04-06 00:56:15,567	WARNING deprecation.py:34 -- DeprecationWarning: `env_index` has been deprecated. Use `episode.env_id` instead. This will rai

checkpoint saved at /home/azibit/ray_results/PPO_unity_env_2021-04-06_00-56-093qzpcrxa/checkpoint_813/checkpoint-813
custom_metrics: {}
date: 2021-04-06_00-56-33
done: false
episode_len_mean: .nan
episode_reward_max: .nan
episode_reward_mean: .nan
episode_reward_min: .nan
episodes_this_iter: 0
episodes_total: 6456
experiment_id: 8282e29d6bd049c5b9713d59cbf0a763
hostname: azibit-Lenovo-Y520-15IKBM
info:
  learner:
    default_policy:
      allreduce_latency: 0.0
      cur_kl_coeff: 0.2
      cur_lr: 5.0e-05
      entropy: 0.9787910431623459
      entropy_coeff: 0.0
      kl: 0.02692586090415716
      policy_loss: -0.05564086697995663
      total_loss: -0.03235322143882513
      vf_explained_var: -1.0
      vf_loss: 0.017902474151924253
  num_steps_sampled: 487800
  num_steps_trained: 487800
iterations_since_restore: 1
node_ip: 192.168.1.4
num_healthy_workers: 1
off_policy_estimator: {}
perf:
  cpu_util_percent: 20.1
  ram_util_percent: 90.4925925925926
pid: 54132
policy_reward_max: {}
p